In [1]:
import os

# spark_version
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-17 22:46:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.66MB/s    in 0.2s    

2021-11-17 22:46:26 (5.66 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# EXTRACT

In [4]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
electronics_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
electronics_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [5]:
# Count the number of records
electronics_df.count()

3093869

# TRANSFORM


In [6]:
# Select "review_id", "customer_id", "product_id", "product_parent" and "review_date" columns
reviews_df = electronics_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviews_df.show(truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2MTG1GCZLR2DK|41409413   |B00428R89M|112201306     |2015-08-31 |
|R2HBOEM8LE9928|49668221   |B000068O48|734576678     |2015-08-31 |
|R1P4RW1R9FDPEE|12338275   |B000GGKOG8|614448099     |2015-08-31 |
|R1EBPM82ENI67M|38487968   |B000NU4OTA|72265257      |2015-08-31 |
|R372S58V6D11AT|23732619   |B00JOQIO6S|308169188     |2015-08-31 |
|R1A4514XOYI1PD|21257820   |B008NCD2LG|976385982     |2015-08-31 |
|R20D9EHB7N20V6|3084991    |B00007FGUF|670878953     |2015-08-31 |
|R1WUTD8MVSROJU|8153674    |B00M9V2RMM|508452933     |2015-08-31 |
|R1QCYLT25812DM|52246189   |B00J3O9DYI|766372886     |2015-08-31 |
|R904DQPBCEM7A |41463864   |B00NS1A0E4|458130381     |2015-08-31 |
|R1DGA6UQIVLKZ7|2781942    |B007B5V092|152891509     |2015-08-31 |
|RLQT3V8SMNIBH |707292     |B00IODHGVG|717335277     |2015-08-

In [7]:
# Drop duplicates and null values
reviews_df = reviews_df.drop_duplicates(["review_id"])
reviews_df = reviews_df.dropna()

In [8]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

# Convert "review_date" column from string to date format
date_format = udf(lambda x: datetime.strptime(x, "%Y-%m-%d"), DateType())
reviews_df = reviews_df.withColumn("review_date", date_format(col("review_date")))

In [9]:
# 'review_id_table' 
reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
# Select "product_id" and "product_title" columns
products_df = electronics_df.select(["product_id","product_title"])

# Drop duplicates and null values
products_df = products_df.drop_duplicates(["product_id"])
products_df = products_df.dropna()

# Show DataFrame
products_df.show(truncate=False)

+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                           |
+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0303532572|TDK Hi8 MP120 Premium Performance Camcorder Videotape (6-pack)                                                                                                                                          |
|0528881469|Rand McNally 528881469 7-inch Intelliroute TND 700 Truck GPS                                                                        

In [11]:
# 'products' table
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [12]:
# unique customers and count of reviews
customers_df = electronics_df.groupBy("customer_id").count()
customers_df = customers_df.withColumnRenamed("count","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|             6|
|   28871066|             1|
|    6711334|             2|
|   46393060|             1|
|   48101124|             1|
|   38209321|             1|
|   47108763|             4|
|   43252205|             1|
|   41283385|             1|
|   46909180|             5|
|    5453476|             2|
|   12406466|             3|
|   48146680|             1|
|    5953456|             1|
|   16457323|             5|
|   24540309|             1|
|   43366878|             2|
|   28796717|             1|
|   28258386|             1|
|   26079415|             2|
+-----------+--------------+
only showing top 20 rows



In [13]:
# 'customers' table
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [14]:
# Select "review_id", "star_rating", "helpful_votes", "total_votes", "vine" columns
vine_df = electronics_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

# Make sure there are only unique id numbers and drop null values
vine_df = vine_df.drop_duplicates(["review_id"])
vine_df = vine_df.dropna()

vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10000WMGXS51T|          5|            1|          1|   N|
|R10003OLR2P5UE|          3|            0|          0|   N|
|R10005O193PJ6W|          3|            2|          2|   N|
|R10008LR7CU84N|          1|            2|          4|   N|
|R10009JN2UWOJC|          5|            0|          0|   N|
|R1000AMVKPW32O|          5|            0|          0|   N|
|R1000CJMO2L8X4|          5|            0|          0|   N|
|R1000EDGJUU3CU|          5|            1|          1|   N|
|R1000EG9XXBLXT|          5|            1|          1|   N|
|R1000GDC9S9VMP|          5|            0|          0|   N|
|R1000K9CAN0EKG|          2|            5|          5|   N|
|R1000T15352LT4|          2|            2|          4|   N|
|R10014QUKXBWJB|          5|            1|          1|   N|
|R1001FSAU03TN7|          5|            

In [15]:
# 'vine_table' 
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# LOAD


In [17]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [18]:
import sys
sys.path.append('/content/gdrive/MyDrive/Amazon Vine Program Analysis')

In [19]:
from config import endpoint, user_name, password
# Configure settings for RDS
mode1 = "append"
mode2 = "overwrite"
jdbc_url="jdbc:postgresql://" + endpoint + ":5432/VineProgramDB"
config = {"user": user_name, 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [20]:
# Write reviews_df to 'review_id_table' table in RDS
reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode1, properties=config)

In [21]:
# Write products_df to 'products' table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode1, properties=config)

In [22]:
# Write customers_df to 'customers' table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode1, properties=config)

In [23]:
# Write vine_df to 'vine_table' table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode1, properties=config)